In [19]:
import keras.backend as K
from keras.applications import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter
# train/test mode  0 or 1
K.set_learning_phase(0)

In [20]:
'''load trained resnet50 model from imagenet'''
model = ResNet50(include_top=True, weights='imagenet')
config = model.get_config()
weights = model.get_weights()
new_model = Model.from_config(config)
new_model.set_weights(weights)

In [13]:
#new_model.summary()

<tf.Tensor 'input_3_1:0' shape=(?, 224, 224, 3) dtype=float32>

In [14]:
print(new_model.input)
print(new_model.output)

Tensor("input_3_1:0", shape=(?, 224, 224, 3), dtype=float32)
Tensor("fc1000_5/Softmax:0", shape=(?, 1000), dtype=float32)


In [18]:
'''model visualization'''
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
#SVG(model_to_dot(new_model).create(prog='dot', format='svg'))

In [3]:
#save model as tf_serving foramt
export_path = '/tmp/resnet50_tmp/1'
builder = saved_model_builder.SavedModelBuilder(export_path)

signature = predict_signature_def(inputs={'images': new_model.input},
                                  outputs={'scores': new_model.output})

with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={'predict': signature})
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/resnet50_tmp/1/saved_model.pb
